In [ ]:
# This code is to split the dataset in to test and train data, and then
# run a random forest algorithm to predice the price of food. 


# Initial tests show that increasing number of trees reaches a min MSE 
# at around 40 trees. 

# Initial testing also showed the split between test and train at 20% 
# test data has given a lower MSE (mean squared error)

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
nutrition = pd.read_pickle("./nutrition_data_clean.pkl")
nutrition.drop(nutrition.columns[1:3], axis=1, inplace=True) 
# removing the brand, and food type columns for data cleaning. 
nutrition

,name,Price (£),Weight (GRAMS),Price per Weight (£/100Gram),Carbon Group,Land use (m2/100g),GHG(kgco2eq/100g),Water use (L/100g),Acidifying emissions(kgSO2eq/100g),Eutr emissions (kg PO43-eq per 100g),Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g)
0,Pillsbury Cinnamon Rolls With Icing Refrigerat...,1.70,260.0,0.653846,Bread products,0.3482,0.1441,56.7,0.001209,0.000706,330,11.27,4.34,53.42,21.34,1.4,0.0,3.250
1,Waffles Buttermilk Frozen Ready-To-Heat,1.50,567.0,0.264550,Bread products,0.3482,0.1441,56.7,0.001209,0.000706,273,9.22,6.58,41.05,4.30,2.2,15.0,1.898
2,Waffle Buttermilk Frozen Ready-To-Heat Toasted,1.50,567.0,0.264550,Bread products,0.3482,0.1441,56.7,0.001209,0.000706,309,9.49,7.42,48.39,4.41,2.6,13.0,2.275
3,Waffle Buttermilk Frozen Ready-To-Heat Microwaved,1.50,567.0,0.264550,Bread products,0.3482,0.1441,56.7,0.001209,0.000706,289,9.40,6.92,44.16,4.50,2.4,16.0,2.057
4,Waffle Plain Frozen Ready-To-Heat Microwave,1.50,567.0,0.264550,Bread products,0.3482,0.1441,56.7,0.001209,0.000706,298,9.91,6.71,45.41,5.04,2.4,16.0,1.580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218,Plantain Fried,0.90,85.0,1.058824,Other vegetables,0.0310,0.0455,8.3,0.000531,0.000186,241,10.16,1.66,40.60,19.10,2.9,0.0,1.507
1219,Romaine Lettuce Raw,1.00,400.0,0.250000,Other vegetables,0.0310,0.0455,8.3,0.000531,0.000186,19,0.27,1.39,3.78,0.71,3.1,0.0,0.053
1220,Palak Paneer,3.75,500.0,0.750000,Cheese,8.0642,2.1240,473.5,0.014894,0.008875,96,6.84,5.23,4.32,1.89,1.2,7.0,1.486
1221,Carrots Raw Salad,0.41,1000.0,0.041000,Other vegetables,0.0310,0.0455,8.3,0.000531,0.000186,208,15.70,1.22,17.17,11.23,2.3,9.0,2.452


In [4]:
nutrition = nutrition.dropna() # dropping NaN values. This ensures random forests can accept data. 
nutrition

,name,Price (£),Weight (GRAMS),Price per Weight (£/100Gram),Carbon Group,Land use (m2/100g),GHG(kgco2eq/100g),Water use (L/100g),Acidifying emissions(kgSO2eq/100g),Eutr emissions (kg PO43-eq per 100g),Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g)
0,Pillsbury Cinnamon Rolls With Icing Refrigerat...,1.70,260.0,0.653846,Bread products,0.3482,0.1441,56.7,0.001209,0.000706,330,11.27,4.34,53.42,21.34,1.4,0.0,3.250
1,Waffles Buttermilk Frozen Ready-To-Heat,1.50,567.0,0.264550,Bread products,0.3482,0.1441,56.7,0.001209,0.000706,273,9.22,6.58,41.05,4.30,2.2,15.0,1.898
2,Waffle Buttermilk Frozen Ready-To-Heat Toasted,1.50,567.0,0.264550,Bread products,0.3482,0.1441,56.7,0.001209,0.000706,309,9.49,7.42,48.39,4.41,2.6,13.0,2.275
3,Waffle Buttermilk Frozen Ready-To-Heat Microwaved,1.50,567.0,0.264550,Bread products,0.3482,0.1441,56.7,0.001209,0.000706,289,9.40,6.92,44.16,4.50,2.4,16.0,2.057
4,Waffle Plain Frozen Ready-To-Heat Microwave,1.50,567.0,0.264550,Bread products,0.3482,0.1441,56.7,0.001209,0.000706,298,9.91,6.71,45.41,5.04,2.4,16.0,1.580
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1218,Plantain Fried,0.90,85.0,1.058824,Other vegetables,0.0310,0.0455,8.3,0.000531,0.000186,241,10.16,1.66,40.60,19.10,2.9,0.0,1.507
1219,Romaine Lettuce Raw,1.00,400.0,0.250000,Other vegetables,0.0310,0.0455,8.3,0.000531,0.000186,19,0.27,1.39,3.78,0.71,3.1,0.0,0.053
1220,Palak Paneer,3.75,500.0,0.750000,Cheese,8.0642,2.1240,473.5,0.014894,0.008875,96,6.84,5.23,4.32,1.89,1.2,7.0,1.486
1221,Carrots Raw Salad,0.41,1000.0,0.041000,Other vegetables,0.0310,0.0455,8.3,0.000531,0.000186,208,15.70,1.22,17.17,11.23,2.3,9.0,2.452


In [5]:
nutrition.dtypes # checking types of data

name                                      object
Price (£)                                float64
Weight (GRAMS)                           float64
Price per Weight (£/100Gram)             float64
Carbon Group                              object
Land use (m2/100g)                       float64
GHG(kgco2eq/100g)                        float64
Water use (L/100g)                       float64
Acidifying emissions(kgSO2eq/100g)       float64
Eutr emissions  (kg PO43-eq per 100g)    float64
Calories                                   int64
Fat (g)                                  float64
Protein (g)                              float64
Carbohydrate (g)                         float64
Sugars (g)                               float64
Fiber (g)                                float64
Cholesterol (mg)                         float64
Saturated Fats (g)                       float64
dtype: object

In [6]:
X = nutrition.iloc[:, 10:].astype('float64').dropna() 
# pulling out the features data
X

,Calories,Fat (g),Protein (g),Carbohydrate (g),Sugars (g),Fiber (g),Cholesterol (mg),Saturated Fats (g)
0,330.0,11.27,4.34,53.42,21.34,1.4,0.0,3.250
1,273.0,9.22,6.58,41.05,4.30,2.2,15.0,1.898
2,309.0,9.49,7.42,48.39,4.41,2.6,13.0,2.275
3,289.0,9.40,6.92,44.16,4.50,2.4,16.0,2.057
4,298.0,9.91,6.71,45.41,5.04,2.4,16.0,1.580
...,...,...,...,...,...,...,...,...
1218,241.0,10.16,1.66,40.60,19.10,2.9,0.0,1.507
1219,19.0,0.27,1.39,3.78,0.71,3.1,0.0,0.053
1220,96.0,6.84,5.23,4.32,1.89,1.2,7.0,1.486
1221,208.0,15.70,1.22,17.17,11.23,2.3,9.0,2.452


In [7]:
y = nutrition.iloc[:, 3:4].astype('float64').dropna()
# pulling out the price data
y

,Price per Weight (£/100Gram)
0,0.653846
1,0.264550
2,0.264550
3,0.264550
4,0.264550
...,...
1218,1.058824
1219,0.250000
1220,0.750000
1221,0.041000


In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)
# splitting the data into train and test data. 
#size =0.2 means 80% data is training data, 20% testing

In [9]:
#implementing the random forest here

from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=10, random_state=0)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

<ipython-input-9-d7e58c9b7a6b>:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(X_train, y_train)


In [16]:
y_train.shape
y_train[:10]
# need to become only the values on the right not the stuff on the left

,Price per Weight (£/100Gram)
788,0.979381
589,0.330033
950,0.875000
565,0.645161
929,1.000000
1071,0.900000
1005,1.400000
731,0.550000
557,0.595238
85,1.219512


In [10]:
# checking the outputs 
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Mean Absolute Error: 0.7977784504395011
Mean Squared Error: 20.10790207490557
Root Mean Squared Error: 4.484183546076763


In [12]:
np.mean(y)
np.std(y)

Price per Weight (£/100Gram)    6.671872
dtype: float64